In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
import os
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")

In [ ]:
%%writefile streamlit_app.py
import os
import streamlit as st
import random
import base64
from PIL import Image
from openai import OpenAI

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


# OpenAI APIを呼び出して応答を取得する関数
def get_chatgpt_response(conversations, image_path):
    with open(image_path, "rb") as image_file:
        image = base64.b64encode(image_file.read()).decode('utf-8')
    client = OpenAI(api_key="api_key")
    response = client.chat.completions.create(model='gpt-4o',
                                              messages=[
                                                  {"role": "developer",
                                                   "content": [
                                                       {"type": "text", "text": "あなたはワンピースに関する知識を持ちません。ワンピースに関する質問は分からないと答え、画像の外見のみから質問に対して「はい」か「いいえ」のみでお答えください。"}
                                                       ],
                                                   },
                                                  {"role": "user",
                                                   "content": [
                                                       {"type": "text", "text": conversations},
                                                       {"type": "image_url",
                                                        "image_url": {"url": f"data:image/png;base64,{image}"}},
                                                       ],
                                                   }])
    return response.choices[0].message.content

# ポケモンのデータ
name_dict = {"Mr.3":["ギャルディーノ","Mr.3","Mr3","ミスタースリー","ミスター3","ミスター３"],
     "アーロン":["アーロン"],
     "アラマキ":["アラマキ","緑牛"],
     "イッショウ":["イッショウ","藤虎"],
     "イヌアラシ":["イヌアラシ"],
     "イワンコフ":["イワンコフ","エンポリオ・イワンコフ","エンポリオイワンコフ"],
     "ウィーブル":["ウィーブル","エドワードウィーブル","エドワード・ウィーブル"],
     "ウソップ":["ウソップ"],
     "ウタ":["ウタ"],
     "ウルージ":["ウルージ"],
     "エース":["エース","ポートガス・Ⅾ・エース","ポートガスⅮエース","ポートガス・D・エース","ポートガスDエース"],
     "エネル":["エネル","ゴッドエネル","ゴッド・エネル"],
     "お玉":["お玉"],
     "おでん":["おでん","光月おでん"],
     "オロチ":["オロチ","黒炭オロチ"],
     "お菊":["お菊","菊之丞","菊の丞","菊ノ丞"],
     "ガープ":["ガープ","モンキー・Ｄ・ガープ","モンキーＤガープ","モンキー・D・ガープ","モンキー・D・ガープ"],
     "カイドウ":["カイドウ"],
     "ガイモン":["ガイモン"],
     "カク":["カク"],
     "カタクリ":["カタクリ","シャーロット・カタクリ","シャーロットカタクリ"],
     "カリファ":["カリファ"],
     "カン十郎":["カン十郎","かん十郎","黒炭カン十郎","黒炭かん十郎"],
     "キッド":["キッド","ユースタス・キッド","ユースタスキッド"],
     "キャロット":["キャロット"],
     "キング":["キング","アルベル"],
     "クイーン":["クイーン","サイエン"],
     "クザン":["クザン","青雉"],
     "クマ":["クマ","くま","バーソロミュー・くま","バーソロミュー・クマ","バーソロミューくま","バーソロミュークマ"],
     "クロコダイル":["クロコダイル","サー・クロコダイル","サークロコダイル"],
     "コビー":["コビー"],
     "サカズキ":["サカズキ","赤犬"],
     "サニー号":["サニー号","サウザンドサニー号","サウザンド・サニー号"],
     "サボ":["サボ"],
     "サンジ":["サンジ","ヴィンスモーク・サンジ","ヴィンスモークサンジ"],
     "シーザー":["シーザー","シーザー・クラウン","シーザークラウン"],
     "シキ":["シキ"],
     "シャンクス":["シャンクス"],
     "しらほし":["しらほし"],
     "ジンベエ":["ジンベエ"],
     "スモーカー":["スモーカー"],
     "ゼット":["ゼット","ゼファー"],
     "センゴク":["センゴク"],
     "ゾロ":["ゾロ","ロロノアゾロ","ロロノア・ゾロ"],
     "たしぎ":["たしぎ"],
     "チャルロス聖":["チャルロス","チャルロス聖"],
     "チョッパー":["チョッパー","トニートニー・チョッパー"],
     "ティーチ":["ティーチ","マーシャル・Ｄ・ティーチ","マーシャルＤティーチ","マーシャル・D・ティーチ","マーシャルDティーチ","黒ひげ","黒髭"],
     "ドフラミンゴ":["ドフラミンゴ","ドンキホーテ・ドフラミンゴ","ドンキホーテドフラミンゴ"],
     "ナミ":["ナミ"],
     "ニューデート":["ニューデート","エドワード・ニューゲート","エドワードニューゲート","白髭","白ひげ"],
     "バギー":["バギー"],
     "ハンコック":["ハンコック","ボアハンコック","ボア・ハンコック"],
     "ビビ":["ビビ","ネフェルタリ・ビビ","ネフェルタリビビ"],
     "フランキー":["フランキー","カティフラム","カティ・フラム"],
     "ブルック":["ブルック"],
     "ペロスペロー":["ペロスペロー","シャーロット・ペロスペロー","シャーロットペロスペロー"],
     "ペローナ":["ペローナ"],
     "ポニー":["ポニー","ジュエリー・ボニー","ジュエリーボニー"],
     "ボルサリーノ":["ボルサリーノ","黄猿"],
     "マゼラン":["マゼラン"],
     "ミホーク":["ミホーク","ジュラキュール・ミホーク","ジュラキュールミホーク"],
     "メリー号":["メリー号","ゴーイングメリー号","ゴーイング・メリー号"],
     "モリア":["モリア","ゲッコー・モリア","ゲッコーモリア"],
     "ヤマト":["ヤマト","やまと"],
     "リンリン":["リンリン","シャーロット・リンリン","シャーロットリンリン","ビッグマム","ビッグ・マム"],
     "ルッチ":["ルッチ","ロブ・ルッチ","ロブルッチ"],
     "ルフィ":["ルフィ","モンキー・D・ルフィ","モンキーDルフィ","モンキー・Ｄ・ルフィ","モンキーＤルフィ"],
     "レイリー":["レイリー","シルバーズ・レイリー","シルバーズレイリー"],
     "レベッカ":["レベッカ"],
     "ロー":["ロー","トラファルガー・ロー","トラファルガーロー","トラファルガー・D・ワーテル・ロー","トラファルガーDワーテルロー","トラファルガー・Ｄ・ワーテル・ロー","トラファルガーＤワーテルロー"],
     "ロジャー":["ロジャー","ゴールド・ロジャー","ゴールドロジャー","ゴール・D・ロジャー","ゴールDロジャー","ゴール・Ｄ・ロジャー","ゴール・Ｄ・ロジャー"],
     "ロビン":["ロビン","ニコ・ロビン","ニコロビン"],
     "錦えもん":["錦えもん","錦衛門","きんえもん"],
     "ネコマムシ":["ネコマムシ"]}

name_list = list(name_dict.keys())

# Streamlit アプリの実行
if __name__ == "__main__":
    st.title("ワンピースアキネーター")

    if "stage" not in st.session_state:
        st.session_state.stage = "start"
        st.session_state.random_number = random.randint(0, len(name_list) - 1)
        st.session_state.messages = []

    # キャラクター選定
    onepiece_name = name_list[st.session_state.random_number]
    onepiece_names = name_dict[onepiece_name]
    image_path = f"/content/drive/MyDrive/ワンピース/{onepiece_name}.jpg"

    if st.session_state.stage == "start":
        st.write("ワンピースのキャラクターについて質問してください。")
        question_raw = st.text_input("質問:")

        if st.button("質問する") and question_raw:  # 文言を追加
            response = get_chatgpt_response(question_raw, image_path)

            # チャット履歴を保存
            st.session_state.messages.append(("あなた", question_raw))
            st.session_state.messages.append(("AI", response))
            st.session_state.stage = "question"
            st.rerun()

    elif st.session_state.stage == "question":
        for role, message in st.session_state.messages:
            st.write(f"**{role}:** {message}")

        if st.button("質問を続ける"):
            st.session_state.stage = "start"
            st.rerun()
        if st.button("解答する"):
            st.session_state.stage = "answer"
            st.rerun()

    elif st.session_state.stage == "answer":
        guess = st.text_input("このキャラクターの名前は？")

        if st.button("答えを確認"):
            st.session_state.show_result = True  # 結果表示フラグをセット

        if getattr(st.session_state, "show_result", False):  # 結果表示フラグが True のとき
            img = Image.open(image_path)
            st.image(img)
            if guess in onepiece_names:
                st.success(f"正解！ {onepiece_name} でした！")
            else:
                st.error(f"不正解... 正解は {onepiece_name} でした！")

            #チャット履歴を表示
            for role, message in st.session_state.messages:
                st.write(f"**{role}:** {message}")

            # "もう一度遊ぶ" & "終了する" ボタンを表示
            if st.button("もう一度遊ぶ"):
                st.session_state.stage = "start"
                st.session_state.random_number = random.randint(0, len(name_list) - 1)
                st.session_state.messages = []
                st.session_state.show_result = False  # 結果表示をリセット
                st.rerun()

            if st.button("終了する"):
                st.write("お疲れさまでした！")

In [ ]:
%%writefile streamlit_app.py
import os
import streamlit as st
import random
from openai import OpenAI

# 無知ネーター用 OpenAI APIを呼び出して応答を取得する関数
def get_chatgpt_response(conversations, url):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-4o',
                                              messages=[
                                                  {"role": "developer",
                                                   "content": [
                                                       {"type": "text", "text": "あなたはポケモンに関する知識を持ちません。ポケモンに関する質問は「分かりません」と答え、画像の外見のみから質問に対して「はい」、「いいえ」、「部分的にそう」のいずれかでお答えください。"}
                                                       ],
                                                   },
                                                  {"role": "user",
                                                   "content": [
                                                       {"type": "text", "text": conversations},
                                                       {"type": "image_url",
                                                        "image_url": {"url": url}},
                                                       ],
                                                   }])
    return response.choices[0].message.content


# 普通のOpenAI APIを呼び出して応答を取得する関数
def get_chatgpt_response2(conversations):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-4o', messages=[{"role": "user", "content": conversations}])
    return response.choices[0].message.content

# ポケモンのデータ
name_list = [
    "フシギダネ", "フシギソウ", "フシギバナ",
    "ヒトカゲ", "リザード", "リザードン",
    "ゼニガメ", "カメール", "カメックス",
    "キャタピー", "トランセル", "バタフリー",
    "ビードル", "コクーン", "スピアー",
    "ポッポ", "ピジョン", "ピジョット",
    "コラッタ", "ラッタ", "オニスズメ", "オニドリル",
    "アーボ", "アーボック", "ピカチュウ", "ライチュウ",
    "サンド", "サンドパン", "ニドラン♀", "ニドリーナ", "ニドクイン",
    "ニドラン♂", "ニドリーノ", "ニドキング", "ピッピ", "ピクシー",
    "ロコン", "キュウコン", "プリン", "プクリン",
    "ズバット", "ゴルバット", "ナゾノクサ", "クサイハナ", "ラフレシア",
    "パラス", "パラセクト", "コンパン", "モルフォン",
    "ディグダ", "ダグトリオ", "ニャース", "ペルシアン",
    "コダック", "ゴルダック", "マンキー", "オコリザル",
    "ガーディ", "ウインディ", "ニョロモ", "ニョロゾ", "ニョロボン",
    "ケーシィ", "ユンゲラー", "フーディン",
    "ワンリキー", "ゴーリキー", "カイリキー",
    "マダツボミ", "ウツドン", "ウツボット",
    "メノクラゲ", "ドククラゲ", "イシツブテ", "ゴローン", "ゴローニャ",
    "ポニータ", "ギャロップ", "ヤドン", "ヤドラン",
    "コイル", "レアコイル", "カモネギ", "ドードー", "ドードリオ",
    "パウワウ", "ジュゴン", "ベトベター", "ベトベトン",
    "シェルダー", "パルシェン", "ゴース", "ゴースト", "ゲンガー",
    "イワーク", "スリープ", "スリーパー",
    "クラブ", "キングラー", "ビリリダマ", "マルマイン",
    "タマタマ", "ナッシー", "カラカラ", "ガラガラ",
    "サワムラー", "エビワラー", "ベロリンガ", "ドガース", "マタドガス",
    "サイホーン", "サイドン", "ラッキー", "モンジャラ",
    "ガルーラ", "タッツー", "シードラ", "トサキント", "アズマオウ",
    "ヒトデマン", "スターミー", "バリヤード", "ストライク",
    "ルージュラ", "エレブー", "ブーバー",
    "カイロス", "ケンタロス", "コイキング", "ギャラドス",
    "ラプラス", "メタモン",
    "イーブイ", "シャワーズ", "サンダース", "ブースター",
    "ポリゴン", "オムナイト", "オムスター", "カブト", "カブトプス",
    "プテラ", "カビゴン",
    "フリーザー", "サンダー", "ファイヤー",
    "ミニリュウ", "ハクリュー", "カイリュー",
    "ミュウツー", "ミュウ"
]

image_url_list = [
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7b705082db2e24dd4ba25166dac84e0a.png",#フシギダネ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6f8144eb4659537733b930d6a299d5a7.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ebccfe6f2ccfe2e851fd29739bf6220c.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/89953014f442146518ef45787d9bb0a4.png",#ヒトカゲ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f11e83606acc0f5a7aa3b7235a373087.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/42062cbeca16839af3efe8a7d61ceb27a30f758b.png",#リザードン
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/17388b7f608671656716ab509ee0fa05.png",#ゼニガメ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7c861520e0eb69f872a4e13a9f04eaba.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/81d2cf3830b75053da5b39dc22aa9a27.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/e97edb1243b8cb641186391489eaf2ee.png",#キャタピー
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f836c66e2fb51a54a2af3af4a7ee7583.png",#トランセル
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4df848214983ca8731ecdcb0ebce9d50.png",#バタフリー
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/1e553bacdabb816e6cc07cf68160bed3.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/e808242baaed1161df1f7361b91b25d8.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/793c7baad51f915c185abe80f6502502.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/9ffa1820513c085c448ee9f884c4991a.png",#ポッポ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5f09d82833710f6c829369835336d7e8.png",#ピジョン
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ad8eb3e08de6bface07c2e390ffb4e86.png",#ピジョット
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7bbce6f5ca4059e34c7b8ca551fe7c08.png",#コラッタ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/dfeb249dd6ad5cf669f01d8174ccbf0e.png",#ラッタ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b836be565d23d99e9899705d9037df72.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/47b8fcfa90ffecb307828c8bfe3f045b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ec956e46baa764b9c939d6838c7becab.png",#アーボ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b6b559df10674cb8da450e67ade5e13f.png",#アーボっく
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5bb0cfd44302cd4df0c0c88d37457931.png",#ピカチュウ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b044779bd695701e03dd7b3bc83421e9.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/fbb27992c96a394d5fda6ba4fbb336f2.png",#3度
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5a8bf38b0f8557538daf3ebdabefc6ed.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/2f075bd7629e4a64e8bccd1f8c7c7d4a.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/71d5b2efd93bd47eb18281a59da3c4c2.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ae056ae20c488428be8ea052b11577b5.png",#に毒院
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5db396d950b721004b2b5e53f56e8b8b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/c314758f6c08f0a33d03c86c7b53173a.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/1e05a1000fbeea6eed06b6e52a429695.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/eaa24f4a7c563fd33ba912c55a7962ba.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/d98dc03a88b9741f32018d27835bad0f.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/513db9297ba760b90348456cb223c796.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/82bee998bb010a5309466084c94c27cd.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/c8e0b40c9ec85d0218754642a64cc047.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/29cc4bfb54379455a36a1c227f99ccf8.png",#プクリン
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/18ba5d304727607f0689f5b31391ae62.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/1898f984daea851560fd8a5d908e3a35.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f013efef989d8ac7c57fad231b19eb59.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/91e20491cfdbac455edffb8bf5ae74c6.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/aed7dc29cf4cfe800648f94ffaac9b2b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/e214554c301a08f064ddeb45e41f95eb.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/acf57b4e1fcf99b5974a5e8289366e7d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/d66185a323be69abb54d5f873667cc78.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/502ec349b1bac1df162fe1cb4a66c964.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ecbbc6139f4008140facd5a3b2f99a36.png",#ディグダ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/e580223a11ba4bc9edcd2e492a1c3371.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/51340f66e3b223cad81f84af8f66e921.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/0fe2142e70d6e3006f3acffeb4d9ebd8.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/2d71923469a502af868dfd337853964a.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5a395498cfc5fc9a01c49ce753e2db12.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f6160acfa612e6f46f6c239b15b8003d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/e704aed22ee77b277d9a0282eab1d54b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/90e53d94baba291ae454514ae51765bb.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b47236a60db344d0ee4764a8a6ae650b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/03e459749bf29e5e75a81e6fce0e4b67.png",#ニョロモ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/8b865ad8fa8d10a78b7133c9ea700ae7.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/3bf0c02c4babfd528cacd453417dd419.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6f85978431106f04ac85dfea54541238.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/332c0e3338ba687eeffc000ac05532c7.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/2b9dbf708bbe9eb6cfe0559507368265.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/42a46a9122314e4a17dd3c1fd33443eb.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4c4f2bba186340436ffe89143e3e01a6.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6ee10ea72fc862fb4ce65cba1da648cd.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4eccda4ac006ff141ac07516a87aa287.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ce836edfeb04623696df2d4c410f733b.png",#ウツドン
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4c86d8dbbb35feb6d5c4f4588d921c91.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/8ad8e9046bf152180985639570c8ad0b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/39e9311c11f99b98d9a8c3f8389470fc.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/02f5389ac72186e9b1b35274ec94dd46.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7c810144066f79406bc5a220e77bf2d4.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/15664c0fd35acf9147024b8687bd0688.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/850922128002557d568af34c82abdb3f.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6144b0af751f7a5047df94a4c36e038a.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f8d806f32ee833db68f00e2c50b136be.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/0b944c561e07117ec2273399acf227fb.png",#ヤドラン
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b7577513f44d55f35305ad586a97fe63.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/9a653e3d65688661381f7f55572f7439.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6e2add6b5fbbe1a16d3cb31344cd8dc1.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7b167dd389fd1b8db1a4c1d245c74761.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/e91182a7d2bf359d5cebfc8bccdb7d77.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/8990153dfc98b2fa981dd841814e96e5.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/0de634ab512dcc62045147798ae0e291.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7b1d23d4f99c5aae3abd25e06b507b9b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ea5a0f05aa038e36609815b59bf5f233.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ca29a426b03aa4a7a5a632ff36db54c7.png",#シェルだー
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/9b1c1410451cae11c74fe0cb0e51542d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b94297db89c9de358e8a982f1e01b249.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/430b0a3b4b9e377d40a23e68f611a683.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/72c82f8be362d1b53ae308d706728411.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/47853714e945a8403fc1edb7e80243ae.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/babe04c8f72839dc6e90edceeb78c56b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/63177e99a2bd182e8c61bc2de43a30ed.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/998ecd61218f0802a045ebb51e596375.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/496ff26477428fbaed6c763ffb1022a2.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/78587c96a7d4d25a3616e7b2f5564006.png",#ビリリダマ
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/2341949cc4a5bdb8675c017c8c29e964.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/596b399e3c5786fc8bf8856adf3f7cf5.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/c6f2d9584d3f8be2baa6f4abb45141be.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/030c9f7e6806afa241394655c25d92a0.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/77c329bd851019496ea393e899a69a7f.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b65b460c5e73b005945b14be54614232.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5e57803d0bd47b21a4ea614298168884.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f7594159023daba4bc87aef29e22ae71.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b9f23ce4b00f0947db1d1e3ca146e5f0.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5ee0e56fc82a7d1f525529e6c52025f3.png",#マタドガス
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b85595d7b46da5177f98c9d5d8e1b985.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/8c014f8be2a1f5a1d65c0f501fd3e113.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/61464042dd1a3e9390602ccb6ab26510.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/f080cddf58f24789e0d55d890362e6ee.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/2cd238818214b37a548a03884422a71d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/5638874bb87667d975adf7dd1117322d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b28d534f2a6ef3c3483a05d445745690.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7c52c66f6681e46c9f75224fea0de440.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/80c2cfb2afe4d843b2fed9d84aea0b5f.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6f9a06aa9a178bffd7584e96c2d458e2.png",#人手まん
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/774a4e2f78322771104d21572102731e.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/61c3e46c087a0bc54775e9b754afc39d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4bfcc6190f6380da093ad509473db6f6.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/aff70cfff745728d89e2ad62b7843078.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4d4680436b10634bb29b0dfab71e3092.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/b5135e364c2102469c4c1779b1bbfe1d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/1d265de7faf17ba529e43e4d5884e903.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/9d8e5856660e43087ceb5856191f4fe7.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/1834616d5ae210cc15dde465ff60afb0.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/c4dc157cd0cf4e2485a81dcb8df91c02.png",#ギャラドス
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/03e89cb38924aefe699902879addeb7e.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/c92e12fa5b843bd8ab8dfe1ed501e577.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/326507f5595907c72d74ac8ea686a6ce.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/185e0940b953a8bbee052540ab80e229.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/3c882f8bd3ada9d4f19a0f15d7cde3e1.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ce7a495289998df2591b160b05db6ab2.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/8ea8bf2182acb6d1a000909a5b90f74b.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/befe7c9979a6ea067502abc57580ea99.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/164b5df92e7f264808a7531f86fca5ae.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/6a9d6abe02eea78cfbd2f057cbb775cc.png",#カブト
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/d67536bf39f592aae27ae097263134f8.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/033f19cea3fd78d491b256dcb9ce2d47.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/4f8f7ac059b4b366f8d6e2875597e277.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/19c50c1cde7493d1aae012b8270a7272.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/2b0afac1febd6fb753b4630dd02d6002.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/453bd551293e6f2ff79332b54bbc46d1.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/a0f99df0f64725e4d39d6fcfdfd03932.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/05ed6fcc75ba76e92db6f2e41125a49f.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/44b8a5591aa246b6a3aa1572481a8c6d.png",
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/ecad7b051d99374b56298a49985c071c.png",#ミュウツー
    "https://zukan.pokemon.co.jp/zukan-api/up/images/index/7659ce2b6e793d5df430205199c7203a.png",
]

# Streamlit アプリの実行
if __name__ == "__main__":
    st.title("ポケモン無知ネーター")

    if "stage" not in st.session_state:
        st.session_state.stage = "start"
        st.session_state.random_number = random.randint(0, len(name_list) - 1)
        st.session_state.messages = []

    # ポケモン選定
    pokemon_name = name_list[st.session_state.random_number]
    pokemon_image = image_url_list[st.session_state.random_number]

    if st.session_state.stage == "start":
        st.write("ポケモンについて何も知らない私がポケモンの写真を見せられています。そのポケモンについて質問して当ててください。")
        question_raw = st.text_input("質問:")

        if st.button("質問する") and question_raw:
            question_modified = f"ポケットモンスターの{pokemon_name}に関して: {question_raw}、\
            そして、解答には{pokemon_name}の名前を含めないこと"  # 文言を追加
            response = get_chatgpt_response(question_modified, pokemon_image)

            # チャット履歴を保存
            st.session_state.messages.append(("あなた", question_raw))
            st.session_state.messages.append(("AI", response))
            st.session_state.stage = "question"
            st.rerun()

    elif st.session_state.stage == "question":
        for role, message in st.session_state.messages:
            st.write(f"**{role}:** {message}")

        if st.button("質問を続ける"):
            st.session_state.stage = "start"
            st.rerun()
        if st.button("解答する"):
            st.session_state.stage = "answer"
            st.rerun()

    elif st.session_state.stage == "answer":
        guess = st.text_input("このポケモンの名前は？")

        if st.button("答えを確認"):
            st.session_state.show_result = True  # 結果表示フラグをセット

        if getattr(st.session_state, "show_result", False):  # 結果表示フラグが True のとき
            st.image(pokemon_image)
            if guess == pokemon_name:
                st.success(f"正解！ {pokemon_name} でした！")
            else:
                st.error(f"不正解... 正解は {pokemon_name} でした！")

            #チャット履歴を表示
            for role, message in st.session_state.messages:
                st.write(f"**{role}:** {message}")

            # AIにポケモンの説明をリクエスト
            description_prompt = f"ポケモンの{pokemon_name} について140文字以内で説明してください。"
            pokemon_description = get_chatgpt_response2(description_prompt)

            # 説明を表示
            st.subheader(f"No.{st.session_state.random_number + 1}: {pokemon_name}")
            st.write(pokemon_description)

            if st.button("ポケモン図鑑を見る"):
                pokemon_url = f"https://zukan.pokemon.co.jp/detail/{str(st.session_state.random_number + 1).zfill(3)}"
                st.markdown(f'[ここを押すとポケモン図鑑に移動するよ]({pokemon_url})', unsafe_allow_html=True)

            # "もう一度遊ぶ" & "終了する" ボタンを表示
            if st.button("もう一度遊ぶ"):
                st.session_state.stage = "start"
                st.session_state.random_number = random.randint(0, len(name_list) - 1)
                st.session_state.messages = []
                st.session_state.show_result = False  # 結果表示をリセット
                st.rerun()

            if st.button("終了する"):
                st.write("お疲れさまでした！")